In [ ]:
from LoadData.loadData import LoadData
from visual.visual import Graph
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

ld = LoadData()
df = ld.loadData('trip').dropna()

vis = Graph()
fig1,fig2 = vis.plot(df1)
fig1.show()
fig2.show()

In [ ]:
df.describe()

# Labels

In [54]:
from labelling.labels import Labels
from labelling.verify_labels import VerifyLabels

# label object
lb_ob = Labels()
vlb_ob = VerifyLabels()

In [ ]:
col_name = 'Vertical'
ev_vals_df = lb_ob.cusum_filter(df,np.sqrt(np.mean(df[col_name]**2))+1.5,col_name)
print(np.sqrt(np.mean(df[col_name]**2)))
print(ev_vals_df.shape)

# Verify Limits

In [ ]:
vlb_ob.verify_samples(df,col_name, ev_vals_df['event'])

In [ ]:
# Set Vertical Limits

vlimits = lb_ob.set_vertical_limits(ev_vals_df)
# print(vlimits.head())

# Set horizontal limits

hlimits = lb_ob.set_horizontal_limits(ev_vals_df)

# Get threshold points

# points above and below h_up and h_dn respectively
thold_pts = lb_ob.get_thold_pts(ev_vals_df,hlimits)
thold_pts

In [ ]:
vlb_ob.verify_horizontal_limits(df,col_name, ev_vals_df,ev_vals_df['event'],vlimits,hlimits)

In [ ]:
# set label according to dataset
# 0 : plain
# -1: pothole
labelled_df = lb_ob.get_labels(ev_vals_df,thold_pts,-1)

In [ ]:
labelled_df['label'].value_counts()

In [ ]:
# Verify labels
vlb_ob.verify_labels(df,col_name,labelled_df,labelled_df['event'],vlimits,hlimits)

In [ ]:
ix = df['Time'].searchsorted(labelled_df['event'])
df['label'] = np.NaN
label_values = labelled_df['label'].values
df.iloc[ix,-1] = label_values
final_df = df.dropna()
final_df.head(10)

# Machine Learning

In [ ]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from model.model import Models

# model object
mdl_ob = Models()

X = df.iloc[:,1:7]
y = df.fillna(0).iloc[:,-1]
print(y.head())

trainX, testX, trainY, testY = train_test_split(X,y, test_size=0.2, shuffle=False)

trainX.shape, trainY.shape

clf = mdl_ob.randomForest(trainX,trainY,n_estimators=200)

pred_y = clf.predict(testX)

accuracy_score(testY,pred_y)

# Saving markers as json

In [ ]:
newdf = final_df[["label","Lat","Long"]].groupby(["Lat","Long"]).mean()
print(newdf.shape)
newdf = newdf[newdf["label"]<-0.8]
print(newdf.shape)
newdf = newdf.reset_index(level=["Lat","Long"]).rename(columns={"Lat":"lat","Long":"long"})
newdf.head(10)

In [ ]:
newdf["label"] = 1
newdf.head()

In [ ]:
js = newdf.to_json(orient="records")
gf = open("ss.txt","w")
gf.write(js)
gf.close()
print(js)